Deocrators replace a function with another callable object, this is a powerful technique for adding funcionailty in a modular, maintainable way.  Let's define a simple function:

In [1]:
def hello():
    "Print a well-known message."
    print('Hello, world')

The preceding function has an attribute called __name__, which is simply the name of the function as the user defined it.

In [2]:
hello.__name__

'hello'

It also has an attribute called __doc__ which is the docstring defined by the user:

In [3]:
hello.__doc__

'Print a well-known message.'

In [4]:
help(hello)

Help on function hello in module __main__:

hello()
    Print a well-known message.



Let's see what happens when we use a decorator on our function.  First let's define a simple no-op decorator:

In [5]:
def noop(f):
    def noop_wrapper():
        return f()

    return noop_wrapper

and decorate our hello function:

In [6]:
@noop
def hello():
    "Print a well-known message."
    print('Hello, world')

Now help() is a whole lot less helpful:

In [7]:
help(hello)

Help on function noop_wrapper in module __main__:

noop_wrapper()



Instead of reporting on the expected hello docstring, we are seeing information about the wrapper function used by the noop decorator.  Looking at the __name__ and __doc__ attributes of the hello function will show why:

In [8]:
hello.__name__

'noop_wrapper'

In [9]:
hello.__doc__

To get the behavior that we are looking for we need to rplace both the __name__ and __doc__ attributes of our noop_wrapper() function with the same attributes from the wrapped hello function:

In [10]:
def noop(f):
    def noop_wrapper():
        return f()
    
    noop_wrapper.__name__ = f.__name__
    noop_wrapper.__doc__ = f.__doc__
    return noop_wrapper

In [11]:
@noop
def hello():
    "Print a well-known message."
    print('Hello, world')

In [12]:
help(hello)

Help on function hello in module __main__:

hello()
    Print a well-known message.



I would be nice if there were a more concise way of creating "wrapper" functions which properly inherited the appropriate attributues from the functions they wrap.

The function wraps() in the functools package does precisely that.  functools.wraps() is itself a decorator-factory which you apply to your wrapper functions.  The wraps() function takes the function to be decorated as its argument, and it returns a decorator that does the hard work of updaing the wrapper function with the wrapped function's attributes.

In [13]:
import functools

def noop(f):
    @functools.wraps(f)
    def noop_wrapper():
        return f()

    return noop_wrapper

In [14]:
@noop
def hello():
    "Print a well-known message."
    print('Hello, world')

In [15]:
help(hello)

Help on function hello in module __main__:

hello()
    Print a well-known message.



In [16]:
hello.__name__

'hello'

In [17]:
hello.__doc__

'Print a well-known message.'

It is probably best to use the functools.wraps() to ensure that your decorated functions continue to behave as your users expect.

Next we are creating a decorator factory, not just a decorator.  A decorator factory is a function that returns a decorator, the actual decorator is customized based on the arguments to the factory.

In [19]:
# A decorator factory: it returns decorators
def check_non_negative(index):
    # This is the actual decorator
    def validator(f):
        # This is the wrapper function
        def wrap(*args):
            if args[index] < 0:
                raise ValueError('Argument {} must be non-negative'.format(index))
            return f(*args)
        return wrap
    return validator

Here is how you can use this decorator to ensure that hte second argument to a function is non-negative:

In [20]:
@check_non_negative(1)
def create_list(value, size):
    return [value] * size

In [21]:
create_list('a', 3)

['a', 'a', 'a']

In [22]:
create_list(123, -6)

ValueError: Argument 1 must be non-negative